In [1]:
import muspy
from io import open
import unicodedata
import string
import re
import random
import numpy as np
import torch
import os
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import sys
sys.path.append("..")
from src.models import *
from src.mmt_dataset import *
import src.representation as representation

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import pickle as pk

In [2]:
qual2idx = {
    'maj': 1,
    'maj6': 1,
    'min': 2,
    'min6': 2,
    'dim': 3,
    'aug': 4,
    '7': 5,
    'maj7': 6,
    'min7': 7,
    'minmaj7': 7,
    'dim7': 8,
    'hdim7': 9,
    'sus2': 10,
    'sus4': 11,
    'sus4(b7)': 11,
    None: 12
}

note2idx = {
    'C': 1,
    'Cb': 12, # 'Cb' is the same as 'B'
    'C#': 2,
    'Db': 2, # 'Db' is the same as 'C#
    'D': 3,
    'D#': 4,
    'Eb': 4, # 'Eb' is the same as 'D#
    'E': 5,
    'Fb': 5, # 'Fb' is the same as 'E'
    'F': 6,
    'E#': 6, # 'E#' is the same as 'F'
    'F#': 7,
    'Gb': 7, # 'Gb' is the same as 'F#
    'G': 8,
    'G#': 9,
    'Ab': 9, # 'Ab' is the same as 'G#'
    'A': 10,
    'A#': 11,
    'Bb': 11, # 'Bb' is the same as 'A#'
    'B': 12,
    'B#': 1, # 'B#' is the same as 'C'
    'N': 13 # 'N' is the same as 'None'
}

def chord2seq(chords, music, target_resolution=12):

    # convert music object tempo list to absolute time
    tempos = [[0,0]]
    prev_tempo = 100
    prev_time = 0
    for tempo in music.tempos:
        tempos.append(((tempo.time - prev_time) * 60 / (prev_tempo * music.resolution) + tempos[-1][0], tempo.qpm))
        prev_tempo = tempo.qpm
        prev_time = tempo.time

    tempos = tempos[1:]

    new_chords = OrderedDict()
    for ix, chord in chords.items():
        off1tmpo = np.where((chord[0] < np.array(tempos)[:, 0]).astype(int).cumsum() == 1)[0]
        chord = (*chord, off1tmpo[0] - 1 if len(off1tmpo) > 0 else len(tempos)-1)
        new_chords[ix] = chord
        # print(chord)

    prev_tempo = 100
    prev_time = 0
    prev_idx = list(new_chords.values())[0][-1]
    metric_chords = OrderedDict()
    for ix, chord in new_chords.items():
        chord_time = chord[0]
        prev_idx = chord[-1]
        cur_tempo = tempos[prev_idx]
        start_beat = (chord_time - cur_tempo[0]) * cur_tempo[1] * music.resolution / 60 + music.tempos[prev_idx].time
        end_beat = (chord[1] - cur_tempo[0]) * cur_tempo[1] * music.resolution / 60 + music.tempos[prev_idx].time
        metric_chords[ix] = (int(start_beat), int(end_beat), chord[2], chord[3])

    seq = []
    for beat, chord in metric_chords.items():
        beat = chord[0]
        dur = chord[1] - chord[0] 
        seq.append((5, round(beat / music.resolution)+1, 1, note2idx[chord[2]], qual2idx[chord[3]], 0))
    return np.array(seq)

In [3]:
pop909_path = "../../POP909-Dataset/POP909/"
out_path = "../data/pop909/"
encoding = representation.get_encoding()

In [23]:
pop909_path = "../../POP909-Dataset/POP909/"
out_path = "../data/pop909/"
encoding = representation.get_encoding()
rep = representation
if not os.path.exists(out_path):
    os.mkdir(out_path)
for fdir in os.listdir(pop909_path):
    if os.path.isdir(pop909_path + fdir):
        mid_path = pop909_path + fdir + f"/{fdir}.mid"
        if os.path.exists(mid_path):
            print(fdir)
            m = muspy.read_midi(mid_path)
        if not os.path.exists(out_path + fdir):
            os.mkdir(out_path + fdir)
        chord_path = pop909_path + fdir + f"/chord_midi.txt"
        if os.path.exists(chord_path):
            print(fdir)
            with open(chord_path, "r") as f:
                lines = f.readlines()
                chords = OrderedDict()
                for ix, line in enumerate(lines):
                    start, end, chord = line.strip("\n").split('\t')
                    start = float(start)
                    end = float(end)
                    chord = chord.split(':') if ':' in chord else chord
                    root = chord[0]
                    quality = chord[1].split("/")[0] if len(chord) > 1 else None
                    chords[ix] = (start, end, root, quality)
        quant_m = muspy.from_music21(m.to_music21().quantize((4,), inPlace=False, recurse=True), 4)
        quant_m.adjust_resolution(m.resolution)
        quant_m.tempos = m.tempos
        chord_seq = chord2seq(chords, quant_m)
        quant_m.adjust_resolution(12)

        m_seq = rep.extract_notes(quant_m, 12)
        enc = rep.encode_notes(m_seq, encoding)
        dec = representation.decode(enc, encoding)

        preamble = np.where(enc[:, 0] == 2)[0][0]
        inter_seq = np.vstack((enc.astype(int)[preamble+1:-1], chord_seq.astype(int)))
        inter_seq = np.stack(sorted(inter_seq, key=lambda x: (x[1], x[2], -x[0])), 0)
        inter_seq = np.concatenate((enc[:preamble+1], inter_seq, enc[-1:]), 0)


        quant_m.tracks = quant_m.tracks[:1]
        mel_sec = rep.extract_notes(quant_m, 12)
        mel_enc = rep.encode_notes(mel_sec, encoding)
        mel_dec = representation.decode(mel_enc, encoding)

        mel_preamble = np.where(mel_enc[:, 0] == 2)[0][0]
        mel_inter_seq = np.vstack((mel_enc.astype(int)[mel_preamble+1:-1], chord_seq.astype(int)))
        mel_inter_seq = np.stack(sorted(mel_inter_seq, key=lambda x: (x[1], x[2], -x[0])), 0)
        mel_inter_seq = np.concatenate((mel_enc[:mel_preamble+1], mel_inter_seq, mel_enc[-1:]), 0)

        chord_d = {int(x[1]-1): [int(y) for y in x[3:]] for x in chord_seq}

        # write everything out
        np.save(out_path + fdir + f"/{fdir}.npy", m_seq)
        np.save(out_path + fdir + f"/{fdir}_enc.npy", enc)
        dec.write_midi(out_path + fdir + f"/{fdir}_dec.mid")
        np.save(out_path + fdir + f"/{fdir}_inter.npy", inter_seq)
        np.save(out_path + fdir + f"/{fdir}_mel.npy", mel_sec)
        np.save(out_path + fdir + f"/{fdir}_mel_enc.npy", mel_enc)
        mel_dec.write_midi(out_path + fdir + f"/{fdir}_mel_dec.mid")
        np.save(out_path + fdir + f"/{fdir}_mel_inter.npy", mel_inter_seq)
        np.save(out_path + fdir + f"/{fdir}_chord.npy", chord_seq)
        np.save(out_path + fdir + f"/{fdir}_notes.npy", m_seq)
        np.save(out_path + fdir + f"/{fdir}_mel_notes.npy", mel_sec)
        with open(out_path + fdir + f"/{fdir}_chord.json", "w") as f:
            json.dump(chord_d, f)


241
241
103
103
702
702
788
788
485
485
058
058
459
459
600
600
052
052
225
225
577
577
252
252
796
796
755
755
549
549
189
189
048
048
431
431
028
028
083
083
120
120
815
815
442
442
158
158
474
474
105
105
443
443
734
734
084
084
714
714
071
071
350
350
133
133
333
333
034
034
222
222
753
753
307
307
021
021
884
884
456
456
655
655
523
523
386
386
675
675
658
658
292
292
861
861
904
904
036
036
851
851
208
208
178
178
676
676
414
414
701
701
854
854
751
751
424
424
466
466
439
439
093
093
248
248
481
481
145
145
094
094
636
636
162
162
179
179
656
656
551
551
605
605
822
822
201
201
191
191
566
566
475
475
900
900
535
535
838
838
087
087
663
663
085
085
164
164
379
379
653
653
180
180
282
282
489
489
569
569
445
445
410
410
786
786
104
104
846
846
265
265
724
724
417
417
890
890
080
080
215
215
554
554
513
513
897
897
217
217
210
210
220
220
173
173
597
597
188
188
433
433
290
290
770
770
321
321
591
591
026
026
398
398
382
382
276
276
013
013
593
593
472
472
680
680
465
465
638
638


In [7]:
# get song names, which are numbered 001 to 909
song_names = []
for i in range(1, 910):
    song_names.append(str(i).zfill(3))

# write out the song names
with open(out_path + "names.txt", "w") as f:
    for name in song_names:
        f.write(name + '/' + name + "\n")

# get the song splits 80:10:10
train_split = song_names[:728]
val_split = song_names[728:819]
test_split = song_names[819:]

# write out the splits
with open(out_path + "train-names.txt", "w") as f:
    for name in train_split:
        f.write(name + '/' + name + "\n")

with open(out_path + "val-names.txt", "w") as f:
    for name in val_split:
        f.write(name + '/' + name + "\n")

with open(out_path + "test-names.txt", "w") as f:
    for name in test_split:
        f.write(name + '/' + name + "\n")